In [1]:
from IPython.display import Image
Image(url='https://vesg.ipsl.upmc.fr/thredds/fileServer/IPSLFS/jservon/CliMAF_Notebooks_html/CliMAF-logo-small.png') 

### A science-oriented framework to ease the analysis of climate model simulations
##### WP5 ANR Convergence
##### Development team: Stéphane Sénési (CNRM-GAME), Gaëlle Rigoudy (CNRM-GAME), Jérôme Servonnat (LSCE-IPSL), Ludivine Vignon (CNRM-GAME), Laurent Franchisteguy (CNRM-GAME), Patrick Brockmann (LSCE-IPSL)
##### Beta-testing: Olivier Marti (LSCE-IPSL), Marie-Pierre Moine (CERFACS), Emilia Sanchez-Gomez (CERFACS)
##   
##### contact: climaf@meteo.fr
##### users list: climaf-users@meteo.fr

# Working with CliMAF on CORDEX ensembles
In this notebook we will see how to make a multi-model ensemble with CORDEX.
Compared with CMIP5 and CMIP6, CORDEX datasets are defined not only by one model but by a GCM/RCM couple (driving_model/model).

1. We will see how to make the most of CliMAF to work on those CORDEX ensembles.
2. and we will see how to access the land/sea mask

#  

## First, import climaf

In [1]:
from climaf.api import *

python => 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]
---
Required softwares to run CliMAF => you are using the following versions/installations:


CliMAF climaf_version = 3.0
CliMAF install => /home/ssenesi/climaf_installs/climaf_running
Cache directory set to : /data/ssenesi/climaf_cache (use $CLIMAF_CACHE if set) 
Cache directory for remote data set to : /data/ssenesi/climaf_cache/remote_data (use $CLIMAF_REMOTE_CACHE if set) 
Available macros read from ~/.climaf.macros are : []
error    : second argument ('curl_tau_atm') must be a script or operator, already declared


## And set verbosity ('critical' -> minimum ; 'debug' -> maximum)

In [4]:
clog('critical') # min verbosity = critical < warning < info < debug = max verbosity

### ... and dont' forget to open the documentation in case you have questions.

### http://climaf.readthedocs.org/

### -> Use the "Quick search" space to search for what you are interested in, it is really powerfull!
#  
#  

## 1. Make a CORDEX ensemble
The specificity of CORDEX is that it works as GCM/RCM couples.
We propose here a way to build a multi-model CORDEX ensemble

In [2]:
clog('debug')

In [14]:
from natsort import natsorted

# -- Let's make a dictionary to specify the general request:
req_dict  = dict(project = 'CORDEX',
                 CORDEX_domain = 'EUR-11',
                 frequency = 'daily',
                 experiment = 'historical',
                 period = '2000',
                 variable = 'tas'
               )

# -- Do the initial request to retrieve all the possible results:
req = ds(model = 'SMHI-RCA4',
         driving_model = '*',
         realization = '*',
         **req_dict)

# -- We will now:
#      - make a loop on all the available 'driving_model' (GCMs)
#      - for each driving_model, we check the model (RCMs) available
#      - and for each GCM/RCM couple, we check the available realizations
#      - store all the matching datasets in myens_dict
#      - and create the ensemble with cens 
myens_dict = dict()

for GCM in req.explore('choices')['driving_model']:
    dumreq = ds(model='*',
                realization = '*',
                driving_model = GCM,
                **req_dict)
    RCMs = dumreq.explore('choices')['model']
    if not isinstance(RCMs, list):
        RCMs = [RCMs]
    for RCM in RCMs:
        dum2req = ds(model=RCM,
                     driving_model = GCM,
                     realization = '*',
                     **req_dict)
        print('GCM = ',GCM,' RCM = ',RCM)
        res_choices = dum2req.explore('choices')
        if res_choices:
            # -- If multiple realizations are available, we take the first one
            if isinstance(res_choices['realization'], list):
                dum2req.kvp['realization'] = natsorted(dum2req.explore('choices')['realization'])[0]
            try:
                myens_dict[GCM+'_'+RCM] = dum2req.explore('resolve')
            except:
                print("              ^--- There are ambiguous facets beyond realization :", end='')  
                for facet in res_choices :
                    if type(res_choices[facet]) is list :
                        print(facet, " = ",res_choices[facet],end='')
                print()
#
my_CORDEX_ens = cens(myens_dict)

GCM =  CCCma-CanESM2  RCM =  CLMcom-CCLM4-8-17
GCM =  CCCma-CanESM2  RCM =  GERICS-REMO2015
GCM =  CCCma-CanESM2  RCM =  SMHI-RCA4
              ^--- There are ambiguous facets beyond realization :model_version  =  ['v1', 'v3']
GCM =  CCCma-CanESM2  RCM =  UCAN-WRF341I
GCM =  CCCma-CanESM2  RCM =  UQAM-CRCM5
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  CLMcom-CCLM4-8-17
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  CNRM-ALADIN53
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  CNRM-ALADIN63
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  DMI-HIRHAM5
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  GERICS-REMO2015
              ^--- There are ambiguous facets beyond realization :model_version  =  ['v1', 'v2']
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  HMS-ALADIN52
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  IPSL-WRF381P
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  KNMI-RACMO22E
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  MOHC-HadREM3-GA7-05
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  RMIB-UGent-ALARO-0
GCM =  CNRM-CERFACS-CNRM-CM5  RCM =  SMHI-RCA4
GCM =  CSIRO-QCCCE-CSIR

In [9]:
clog('critical')

In [10]:
summary(my_CORDEX_ens)

Keys - values:
{'project': 'CORDEX', 'simulation': '', 'variable': 'tas', 'period': 2000, 'domain': 'global', 'root': '/bdd', 'model': 'CLMcom-CCLM4-8-17', 'CORDEX_domain': 'EUR-11', 'model_version': 'v1', 'frequency': 'daily', 'driving_model': 'CCCma-CanESM2', 'realization': 'r1i1p1', 'experiment': 'historical', 'version': 'latest', 'institute': 'CLMcom'}
-- Ensemble members:
CCCma-CanESM2_CLMcom-CCLM4-8-17
/bdd/CORDEX/output/EUR-11/CLMcom/CCCma-CanESM2/historical/r1i1p1/CLMcom-CCLM4-8-17/v1/day/tas/v20171121/tas_EUR-11_CCCma-CanESM2_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_day_19960101-20001231.nc
--
CCCma-CanESM2_GERICS-REMO2015
/bdd/CORDEX/output/EUR-11/GERICS/CCCma-CanESM2/historical/r1i1p1/GERICS-REMO2015/v1/day/tas/v20170329/tas_EUR-11_CCCma-CanESM2_historical_r1i1p1_GERICS-REMO2015_v1_day_19960101-20001231.nc
--
CCCma-CanESM2_UCAN-WRF341I
/bdd/CORDEX/output/SAM-44/UCAN/CCCma-CanESM2/historical/r1i1p1/UCAN-WRF341I/v2/day/tas/v20161020/tas_SAM-44_CCCma-CanESM2_historical_r1i1p1_UCA

/bdd/CORDEX/output/EUR-44/IPSL-INERIS/IPSL-IPSL-CM5A-MR/historical/r1i1p1/IPSL-INERIS-WRF331F/v1/day/tas/v20140301/tas_EUR-44_IPSL-IPSL-CM5A-MR_historical_r1i1p1_IPSL-INERIS-WRF331F_v1_day_19960101-20001231.nc
--
IPSL-IPSL-CM5A-MR_IPSL-WRF381P
/bdd/CORDEX/output/EUR-11/IPSL/IPSL-IPSL-CM5A-MR/historical/r1i1p1/IPSL-WRF381P/v1/day/tas/v20190919/tas_EUR-11_IPSL-IPSL-CM5A-MR_historical_r1i1p1_IPSL-WRF381P_v1_day_19960101-20001231.nc
--
IPSL-IPSL-CM5A-MR_KNMI-RACMO22E
/bdd/CORDEX/output/EUR-11/KNMI/IPSL-IPSL-CM5A-MR/historical/r1i1p1/KNMI-RACMO22E/v1/day/tas/v20191029/tas_EUR-11_IPSL-IPSL-CM5A-MR_historical_r1i1p1_KNMI-RACMO22E_v1_day_19960101-20001231.nc
--
MIROC-MIROC5_CLMcom-CCLM4-8-17
/bdd/CORDEX/output/EUR-11/CLMcom/MIROC-MIROC5/historical/r1i1p1/CLMcom-CCLM4-8-17/v1/day/tas/v20171121/tas_EUR-11_MIROC-MIROC5_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_day_19960101-20001231.nc
--
MIROC-MIROC5_GERICS-REMO2009
/bdd/CORDEX/output/AFR-44/GERICS/MIROC-MIROC5/historical/r1i1p1/GERICS-REMO2009/v1/d

/bdd/CORDEX/output/EUR-11/KNMI/MPI-M-MPI-ESM-LR/historical/r1i1p1/KNMI-RACMO22E/v1/day/tas/v20190619/tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r1i1p1_KNMI-RACMO22E_v1_day_19960101-20001231.nc
--
MPI-M-MPI-ESM-LR_MOHC-HadREM3-GA7-05
/bdd/CORDEX/output/EUR-11/MOHC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MOHC-HadREM3-GA7-05/v1/day/tas/v20200330/tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r1i1p1_MOHC-HadREM3-GA7-05_v1_day_19960101-20001231.nc
--
MPI-M-MPI-ESM-LR_MPI-CSC-REMO2009
/bdd/CORDEX/output/AFR-44/MPI-CSC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MPI-CSC-REMO2009/v1/day/tas/v20160412/tas_AFR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1_day_19960101-20001231.nc
/bdd/CORDEX/output/EUR-11/MPI-CSC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MPI-CSC-REMO2009/v1/day/tas/v20160419/tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1_day_19960101-20001231.nc
/bdd/CORDEX/output/EUR-44/MPI-CSC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MPI-CSC-REMO2009/v1/day/tas/v20150609/tas_EUR-44_MPI-M-MPI-ESM-LR_hi

## 2. Access the mask and land/area fraction sftlf: r0i0p0 and fx

In [6]:
from climaf.api import *

In [11]:
req_dict  = dict(project = 'CORDEX',
                 CORDEX_domain = 'EUR-11',
                 frequency = 'fx',
                 experiment = '*',
                 period = 'fx',
                 realization = 'r0i0p0',
                 variable = 'sftlf'
               )

# -- Do the initial request to retrieve all the possible results:
req = ds(model = '*',
         driving_model = '*',
         **req_dict)

req.explore('choices')

{'model': ['CLMcom-CCLM4-8-17',
  'CLMcom-ETH-COSMO-crCLIM-v1-1',
  'GERICS-REMO2015',
  'IPSL-INERIS-WRF331F',
  'MOHC-HadGEM3-RA',
  'MOHC-HadRM3P',
  'MPI-CSC-REMO2009',
  'SMHI-RCA4',
  'UCAN-WRF341I',
  'UHOH-WRF361H'],
 'model_version': ['v1', 'v1a', 'v2', 'v3'],
 'driving_model': ['CCCma-CanESM2',
  'CNRM-CERFACS-CNRM-CM5',
  'CSIRO-QCCCE-CSIRO-Mk3-6-0',
  'ECMWF-ERAINT',
  'ICHEC-EC-EARTH',
  'IPSL-IPSL-CM5A-LR',
  'IPSL-IPSL-CM5A-MR',
  'MIROC-MIROC5',
  'MOHC-HadGEM2-ES',
  'MPI-M-MPI-ESM-LR',
  'NCC-NorESM1-M',
  'NOAA-GFDL-GFDL-ESM2G',
  'NOAA-GFDL-GFDL-ESM2M'],
 'experiment': ['evaluation', 'historical', 'rcp26', 'rcp45', 'rcp85'],
 'institute': ['CLMcom',
  'CLMcom-ETH',
  'GERICS',
  'IPSL-INERIS',
  'MOHC',
  'MPI-CSC',
  'SMHI',
  'UCAN',
  'UHOH']}

### And of course, you can apply all the pretreatments you want to the members of your ensemble before getting your list of files.
For the available operators, we invite you to go deeper in CliMAF documentation and other CliMAF notebooks.


#   
## This was how to work with CliMAF ensembles!
##  
##  